<a href="https://colab.research.google.com/github/bamchow/Anupam/blob/main/Heston__compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import scipy
from scipy.stats import norm
from scipy.integrate import quad

In [36]:
msft = yf.Ticker("NVDA")
hist = msft.history(period="10y")
hist.drop(["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], axis = 1, inplace = True )
hist.rename(columns = {"Close":"S"}, inplace = True)
hist

,S
Date,
2014-06-23 00:00:00-04:00,0.445269
2014-06-24 00:00:00-04:00,0.438367
2014-06-25 00:00:00-04:00,0.442889
2014-06-26 00:00:00-04:00,0.436939
2014-06-27 00:00:00-04:00,0.437415
...,...
2024-06-14 00:00:00-04:00,131.880005
2024-06-17 00:00:00-04:00,130.979996
2024-06-18 00:00:00-04:00,135.580002


In [37]:
hist["Q"] = hist["S"]/hist["S"].shift(1)
hist.dropna(axis = 0, inplace = True)
hist

,S,Q
Date,,
2014-06-24 00:00:00-04:00,0.438367,0.984500
2014-06-25 00:00:00-04:00,0.442889,1.010315
2014-06-26 00:00:00-04:00,0.436939,0.986566
2014-06-27 00:00:00-04:00,0.437415,1.001090
2014-06-30 00:00:00-04:00,0.441223,1.008705
...,...,...
2024-06-14 00:00:00-04:00,131.880005,1.017514
2024-06-17 00:00:00-04:00,130.979996,0.993176
2024-06-18 00:00:00-04:00,135.580002,1.035120


In [38]:
arr = np.array(hist['Q'])
from scipy import stats
mews = []
for i in range(0, 5):
  mews.append(np.mean(arr**(i+1)))
mews



[1.0026895024789568,
 1.006279283241028,
 1.0107947916720745,
 1.016271398012241,
 1.0227557735562864]

In [39]:
from scipy.optimize import minimize

# Given equations as functions of k, r, sigma, theta
def equations(vars):
    k, r, sigma, theta = vars
    mu1 = 1 + r
    mu2 = (r + 1)**2 + theta
    mu4 = (1 / (k * (k - 2))) * (
        k**2 * r**4 + 4 * k**2 * r**3 + 6 * k**2 * r**2 * theta - 2 * k * r**4 + 6 * k**2 * r**2 + 12 * k**2 * r * theta
        + 3 * k**2 * theta**2 - 8 * k * r**3 - 12 * k * r * theta + 4 * k**2 * r + 6 * k**2 * theta - 12 * k * r**2
        - 24 * k * r * theta - 6 * k * theta**2 - 3 * sigma**2 * theta + k**2 - 8 * k * r - 12 * k * theta - 2 * k
    )
    mu5 = (1 / (k * (k - 2))) * (
        k**2 * r**5 + 5 * k**2 * r**4 + 10 * k**2 * r**3 * theta - 2 * k * r**5 + 10 * k**2 * r**3 + 30 * k**2 * r**2 * theta
        + 15 * k**2 * r * theta**2 - 10 * k * r**4 - 20 * k * r**3 * theta + 10 * k**2 * r**2 + 30 * k**2 * r * theta + 15 * k**2 * theta**2
        - 20 * k * r**3 - 60 * k * r**2 * theta - 30 * k * r * theta**2 - 15 * sigma**2 * theta + 5 * k**2 * r + 10 * k**2 * theta
        - 20 * k * r**2 - 60 * k * r * theta - 30 * k * theta**2 - 15 * sigma**2 * theta + k**2 - 10 * k * r - 20 * k * theta - 2 * k
    )
    return mu1, mu2, mu4, mu5

# Objective function for optimization
def objective(vars):
    k, r, sigma, theta = vars
    mu1, mu2, mu4, mu5 = equations(vars)
    target_mu1 = mews[0]
    target_mu2 = mews[1]
    target_mu4 = mews[3]
    target_mu5 = mews[4]
    return (mu1 - target_mu1)**2 + (mu2 - target_mu2)**2 + (mu4 - target_mu4)**2 + (mu5 - target_mu5)**2

# Constraints
constraints = (
    {'type': 'ineq', 'fun': lambda vars: vars[2]**2},  # sigma^2 > 0
    {'type': 'ineq', 'fun': lambda vars: 2 * vars[0] * vars[3] - vars[2]**2}  # 2k*theta > sigma^2
)

# Initial guess
initial_guess = [1, 1, 1, 1]

# Solve the optimization problem
result = minimize(objective, initial_guess, constraints=constraints)

if result.success:
    k_opt, r_opt, sigma_opt, theta_opt = result.x
    print(f'Optimal values: k = {k_opt:.6f}, r = {r_opt:.6f}, sigma = {sigma_opt:.6f}, theta = {theta_opt:.6f}')
else:
    print('Optimization failed.')



Optimal values: k = 1.749032, r = 0.002411, sigma = 0.006171, theta = 0.001095


In [43]:
import numpy as np
from math import sqrt

i = complex(0, 1)

def c_fun(u, S_t, K, r, tau, kappa, theta, sigma, rho, v_t):
    M = np.sqrt((rho * sigma * 1j * u - kappa)**2 + sigma**2 * (u * 1j + u**2))
    N = (rho * sigma * 1j * u - kappa - M) / (rho * sigma * 1j * u - kappa + M)
    A = r * 1j * u * tau + (kappa * theta) / sigma**2 * (-(rho * sigma * 1j * u - kappa - M) * tau - 2 * np.log((1 - N * np.exp(-M * tau)) / (1 - N)))
    C = ((rho * sigma * 1j * u - kappa - M) * (np.exp(M*tau) - 1)) / (sigma**2 * (1 - N * np.exp(M * tau)))
    heston_cf = np.exp(A + C * v_t + 1j * u * np.log(S_t))

    return heston_cf

def heston_call_price(St, K, r, tau, kappa, theta, sigma, rho, vt):
    P1 = 0
    iterations = 1200
    maxNumber = 100
    ds = maxNumber / iterations

    for j in range(1, iterations):
        s1 = ds * (2 * j + 1) / 2
        s2 = s1 - i
        numerator1 = c_fun(s2, St, K, r, tau, kappa, theta, sigma, rho, vt)
        numerator2 = K * c_fun(s1, St, K, r, tau, kappa, theta, sigma, rho, vt)
        denominator = np.exp(np.log(K) * i * s1) * i * s1

        P1 += ds * (numerator1 - numerator2) / denominator

    P1 = np.real(P1 / np.pi)
    P2 = P1

    call_price = St * (0.5 + P1) - K * np.exp(-r * tau) * (0.5 + P2)

    return call_price


In [57]:
# Calculate logarithmic returns
hist['log_ret'] = np.log(hist['S'] / hist['S'].shift(1))
# Calculate volatility as the difference between latest log return and mean

mean = hist['log_ret'].mean()       # Mean log return
latest = hist['log_ret'].iloc[-1]  # Latest log return
volatility = latest - mean

S_t = hist.iloc[-1]['S']
K = 121
r = ( ( 1 + r_opt)**252) - 1 # Adjusted for annualization
tau = 4/365
kappa = k_opt
theta = theta_opt
sigma = sigma_opt * sqrt(252)  # Adjusted for annualization
rho = 0.8
v_t = volatility

call_price = heston_call_price(S_t, K, r, tau, kappa, theta, sigma, rho, v_t)
print(f"Heston Call Price: {call_price}")

Heston Call Price: 15.036340183472817


In [58]:
import numpy as np
from scipy.stats import norm

def black_scholes_call_price(S_t, K, tau, r, v_t):
    """
    Calculate the Black-Scholes European call option price.

    Parameters:
    S_t (float): Current stock price
    K (float): Strike price
    tau (float): Time to maturity (in years)
    r (float): Risk-free interest rate
    sigma (float): Volatility of the underlying asset

    Returns:
    float: Call option price
    """
    d1 = (np.log(S_t / K) + (r + 0.5 * v_t**2) * tau) / (v_t* np.sqrt(tau))
    d2 = d1 - v_t * np.sqrt(tau)

    call_price = (S_t * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2))
    return call_price


call_price_bs = black_scholes_call_price(S_t, K, tau, r, v_t)
print(f'Black-Scholes Call Price: {call_price_bs:.6f}')


Black-Scholes Call Price: -0.000000
